# Alpha-based crypto quant trading strategy

In [ ]:
import vectorbt as vbt

## Setup

Download data and save to disk.

In [ ]:
timeframe = "1h"
start_date = "2023-01-01"
end_date = "2024-12-31"
symbols = [
    "SOL/USDT:USDT",
    "ETH/USDT:USDT",
    "BTC/USDT:USDT",
    "XRP/USDT:USDT",
    "ZEC/USDT:USDT",
    "DOGE/USDT:USDT",
    "DASH/USDT:USDT",
    "LTC/USDT:USDT",
    "ADA/USDT:USDT",
    "AVAX/USDT:USDT",
    "TRUMP/USDT:USDT",
    "GIGGLE/USDT:USDT",
    "LINK/USDT:USDT",
    "SUI/USDT:USDT",
]
data = vbt.CCXTData.download(symbols, timeframe=timeframe, start=start_date, end=end_date)
data.save(f"crypto_data_{timeframe}_{start_date}_{end_date}.pkl")

Load data from disk to prevent download data frequently.

In [ ]:
data = vbt.Data.load(f"crypto_data_{timeframe}_{start_date}_{end_date}.pkl")